Round 1

In [141]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
import math
import matplotlib.pyplot as plt
from statsmodels.tsa.vector_ar.vecm import coint_johansen
import numpy as np
from pymle.sim.Simulator1D import Simulator1D
from pymle.core.TransitionDensity import ExactDensity, KesslerDensity
from pymle.fit.AnalyticalMLE import AnalyticalMLE
from pymle.models import GeometricBM, OrnsteinUhlenbeck
from sklearn.linear_model import LinearRegression
from scipy.optimize import brute
from decimal import Decimal, ROUND_HALF_UP
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm
from statsmodels.tsa.statespace.structural import UnobservedComponents
from scipy.interpolate import UnivariateSpline
from statsmodels.graphics.tsaplots import plot_acf
from scipy.signal import periodogram, find_peaks
import itertools
from scipy.optimize import minimize, Bounds, LinearConstraint

In [142]:
import itertools

# List of instruments (sn, p, si, se)
instruments = ["sn", "p", "si", "se"]

# Trade prices matrix [from][to]
prices = {
    "sn": {"sn": 1,   "p": 1.45, "si": 0.52, "se": 0.72},
    "p":  {"sn": 0.7, "p": 1,    "si": 0.31, "se": 0.48},
    "si": {"sn": 1.95, "p": 3.1, "si": 1,   "se": 1.49},
    "se": {"sn": 1.34, "p": 1.98, "si": 0.64, "se": 1},
}

def simulate_trade(path, start_amount=1):
    amount = start_amount
    for i in range(len(path) - 1):
        frm = path[i]
        to = path[i + 1]
        amount *= prices[frm][to]
    return amount

best_path = None
best_amount = 0

start = "se"
end = "se"
middle_steps = 4  # total path length = 6 (start + 4 trades + end)

for middle in itertools.product(instruments, repeat=middle_steps):
    full_path = [start] + list(middle) + [end]
    amount = simulate_trade(full_path)
    if amount > best_amount:
        best_amount = amount
        best_path = full_path

print("Best path:", " -> ".join(best_path))
print("Final amount:", best_amount)

Best path: se -> sn -> si -> p -> sn -> se
Final amount: 1.08868032


Round 2

In [143]:
containers = [
    (10, 1),  # Top-left
    (80, 6),  # Top-center-left
    (37, 3),  # Top-center-right
    (17, 1),  # Top-right
    (90, 10), # Mid-left
    (31, 2),  # Mid-center
    (50, 4),  # Mid-right
    (20, 2),  # Bottom-left
    (73, 4),  # Bottom-center
    (89, 8)   # Bottom-right
]

arr_containers = np.array(containers)

# Print the array
print(arr_containers)

p_threshold = np.zeros(arr_containers.shape[0])

B = 10000 
F = 50000
N_choices = 2500

[[10  1]
 [80  6]
 [37  3]
 [17  1]
 [90 10]
 [31  2]
 [50  4]
 [20  2]
 [73  4]
 [89  8]]


In [144]:
for i in range(arr_containers.shape[0]): # Loop through each of the 10 containers
    multiplier = arr_containers[i, 0]
    inhabitants = arr_containers[i, 1]
    
    numerator = (multiplier * B) - (F * inhabitants)
    denominator = N_choices * F
    
    max_pk_for_profit = max(0, numerator / denominator) 
    p_threshold[i] = max_pk_for_profit
    
    print(f"  Container ({multiplier:3d}x, {inhabitants:2d} inh): Max share pk = {max_pk_for_profit:.4f} ({max_pk_for_profit*100:.2f}%)")

  Container ( 10x,  1 inh): Max share pk = 0.0004 (0.04%)
  Container ( 80x,  6 inh): Max share pk = 0.0040 (0.40%)
  Container ( 37x,  3 inh): Max share pk = 0.0018 (0.18%)
  Container ( 17x,  1 inh): Max share pk = 0.0010 (0.10%)
  Container ( 90x, 10 inh): Max share pk = 0.0032 (0.32%)
  Container ( 31x,  2 inh): Max share pk = 0.0017 (0.17%)
  Container ( 50x,  4 inh): Max share pk = 0.0024 (0.24%)
  Container ( 20x,  2 inh): Max share pk = 0.0008 (0.08%)
  Container ( 73x,  4 inh): Max share pk = 0.0042 (0.42%)
  Container ( 89x,  8 inh): Max share pk = 0.0039 (0.39%)


In [145]:
def maximin_1(container_array, B):
    max_metric_val = -float('inf')
    argmax = []
    for i in range(len(container_array)): 
        mult, inhab = container_array[i] 
        metric_val = mult / (inhab + 100) 
        if math.isclose(metric_val, max_metric_val):
            argmax.append((mult, inhab))
        elif metric_val > max_metric_val:
            argmax = [(mult, inhab)]
            max_metric_val = metric_val
    max_profit_val = max_metric_val * B
    return argmax, max_profit_val

In [ ]:
def _objective_f_2d(p_vec, m1, i1, m2, i2):
    p1, p2 = p_vec
    # Add epsilon for numerical stability
    eps = 1e-9
    # Assume i1, i2 are >= 1 based on container data
    term1 = m1 / (i1 + 100 * p1 + eps)
    term2 = m2 / (i2 + 100 * p2 + eps)
    return term1 + term2

def maximin_2(container_array, B, F):
    max_min_profit = -float('inf') 
    best_pairs = [] 

    # Bounds: p1 >= 0, p2 >= 0 (upper bound 1 is implicitly handled by p1+p2<=1)
    bounds = Bounds([0.0, 0.0], [1.0, 1.0]) 
    # Linear Constraint: p1 + p2 <= 1 --> 1*p1 + 1*p2 <= 1
    linear_constraint = LinearConstraint([[1, 1]], [-np.inf], [1.0]) 

    for (m1, i1), (m2, i2) in itertools.combinations(container_array, 2):
        # Initial guess for the optimizer (e.g., start near feasible center)
        initial_guess = [0.5, 0.5] 
        # Ensure guess respects sum constraint if starting there is important
        if sum(initial_guess) > 1: initial_guess = [0.5/sum(initial_guess), 0.5/sum(initial_guess)]

        # Perform 2D constrained minimization
        result = minimize(
            _objective_f_2d, 
            initial_guess, 
            args=(m1, i1, m2, i2), # Pass fixed M, I parameters
            method='SLSQP',
            bounds=bounds, 
            constraints=[linear_constraint]
        )
        
        min_summed_ratio = result.fun 
            
        # Calculate the maximin profit for this pair
        current_min_profit = (min_summed_ratio * B) - F 
        
        # Update tracker for the overall maximum maximin profit
        current_pair_tuple = tuple(sorted([(m1, i1), (m2, i2)]))
        if math.isclose(current_min_profit, max_min_profit):
             if current_pair_tuple not in {tuple(p) for p in best_pairs}:
                 best_pairs.append(list(current_pair_tuple))
        elif current_min_profit > max_min_profit:
            best_pairs = [list(current_pair_tuple)] 
            max_min_profit = current_min_profit
            
    final_best_pairs = [tuple(pair) for pair in best_pairs]
    if max_min_profit == -float('inf'): return [], -float('inf') 
    return final_best_pairs, max_min_profit

# def maximin_2(container_array, B, F):
#     """
#     Solves the maximin optimization problem for choosing TWO containers,
#     using the SHARE-BASED payoff model V = M*B / (I + 100*p).
#     Assumes the worst case is p1=0,p2=1 OR p1=1,p2=0.
#     Simplified version without error handling.
#     """
#     max_profit = -float('inf') 
#     best_pairs = [] 
    
#     # Ensure data is iterable (works for list of tuples or numpy array)
#     data_iterable = container_array
#     if isinstance(container_array, np.ndarray):
#          data_iterable = [tuple(row) for row in container_array]

#     # Iterate through all unique pairs of containers
#     for (m1, i1), (m2, i2) in itertools.combinations(data_iterable, 2):
        
#         # Assume inhabitants i1, i2 are >= 1 based on container data
        
#         # Calculate the summed ratios for the two extreme adversarial allocations
#         # Case A: Adversary sets p1=0, p2=1 (maximizes denominator for container 2)
#         # Profit ratio component from C1 is M1/I1, from C2 is M2/(I2+100)
#         ratio_A = (m1 / i1) + (m2 / (i2 + 100)) 
        
#         # Case B: Adversary sets p1=1, p2=0 (maximizes denominator for container 1)
#         # Profit ratio component from C1 is M1/(I1+100), from C2 is M2/I2
#         ratio_B = (m1 / (i1 + 100)) + (m2 / i2) 
        
#         # Adversary chooses the case that minimizes the combined payoff ratio
#         min_summed_ratio = min(ratio_A, ratio_B)
        
#         # Calculate the total profit for this pair in the worst case, including fee
#         profit = (min_summed_ratio * B) - F 
        
#         # Update max profit and best pair(s)
#         # Use tuples for comparison/storage, sort pair for consistency
#         current_pair_tuple = tuple(sorted([(m1, i1), (m2, i2)])) 
#         if math.isclose(profit, max_profit):
#              # Simple append, ignore rare duplicate possibility for brevity
#              if current_pair_tuple not in {tuple(sorted(p)) for p in best_pairs}:
#                  best_pairs.append(list(current_pair_tuple))
#         elif profit > max_profit:
#             best_pairs = [list(current_pair_tuple)] 
#             max_profit = profit
            
#     # Convert inner lists back to tuples for standard output format
#     final_best_pairs = [tuple(pair) for pair in best_pairs]
    
#     return final_best_pairs, max_profit

In [152]:
maximin1_choice, maximin1_profit = maximin_1(arr_containers, B)
maximin2_choice, maximin2_profit = maximin_2(arr_containers, B, F)

print(f"Maximin choice (1 container): {maximin1_choice}")
print(f"Maximin profit (worst-case, 1 container): {maximin1_profit:.2f}")
print("-" * 30)
print(f"Maximin choice(s) (2 containers): {maximin2_choice}")
print(f"Maximin profit (worst-case, 2 containers): {maximin2_profit:.2f}")
print("-" * 30)

Maximin choice (1 container): [(89, 8)]
Maximin profit (worst-case, 1 container): 8240.74
------------------------------
Maximin choice(s) (2 containers): [((89, 8), (90, 10))]
Maximin profit (worst-case, 2 containers): -19661.25
------------------------------
